# Compute seasonal mean

In [1]:
def ComputeSeasonalMean(DateArr,idata,season='DJF'):
	"""
	INPUT: 
        DateArr [time] (list of datetime objects)
        idata [lats,lons,time]
        season='DJF'
	OUPTUT: DateArr_season, odata
	"""

	import pandas as pd
	import numpy as np
	import matplotlib.pylab as plt

	Months = 'NDJFMAMJJASOND'
	DaysPerMonth = np.array([30,31,31,28,31,30,31,30,31,31,30,31,30,31])
	IndexMonth = np.array([11,12,1,2,3,4,5,6,7,8,9,10,11,12])
		
	nmonth = len(season)
	nh = int((nmonth-1)//2)
		
	idata = np.ma.array(idata)
	
	if idata.ndim == 1:
		idata = idata[np.newaxis,np.newaxis,:]
		
	time_dim = idata.shape[2]		
		
	if len(DateArr) != time_dim:
		print('ERROR! Inconsistent lengths.')
		return False	
		
	if nmonth % 2 == 0:
		print('ERROR! Wrong length of season. Season must include an odd number of months.')
		return False	

	try:
		DPM_season = DaysPerMonth[Months.index(season):Months.index(season)+nmonth] 
		central_month = IndexMonth[Months.index(season)+nh]
	except:
		print('ERROR! Undefined season.')
		return False


	DateArr_season = []
	for i in range(time_dim):
		if DateArr[i].month == central_month:
			DateArr_season += [DateArr[i]]		

	odata = np.ma.zeros([idata.shape[0],idata.shape[1],len(DateArr_season)])
	cnto = 0
	
	for i in range(time_dim):
		if 'F' in season:
			if DateArr[i].year%4 == 0:
				DPM_season[season.index('F')] = 29
			else:
				DPM_season[season.index('F')] = 28
		    
		if DateArr[i].month == central_month:	
			if i == 0:
				odata[:,:,cnto] = np.sum(idata[:,:,i:i+nh+1]*DPM_season[nh:],axis=2)/np.sum(DPM_season[nh:])
			elif i == time_dim - 1:
				odata[:,:,cnto] = np.sum(idata[:,:,i-nh:i+1]*DPM_season[:nh+1],axis=2)/np.sum(DPM_season[:nh+1])
			else:
				odata[:,:,cnto] = np.sum(idata[:,:,i-nh:i+nh+1]*DPM_season[:],axis=2)/np.sum(DPM_season)
				
			cnto += 1 

	odata = np.squeeze(odata)
	return DateArr_season, odata

# Compute significance

In [2]:
def ComputeSignificanceArray(idata, DateArr, CL=0.95, factor=120, ano=False, ac='ar1'):
    # nicht für wechselnde Maske
    # Anomalien werden berechnet, dh es können auch volle Daten in idata übergeben werden
    
    import numpy as np
    import scipy.stats as stats
    from sklearn import linear_model
    
    full_shape = idata.shape
    mask = idata[:,:,0].mask # maske wird nur von 1. zeitschritt übernommen!!!
    index = np.where(mask == False)
    
    idata = idata[index][:,np.newaxis,:]
    shape = idata.shape

    if ano == False:
        anomalies, _ = ComputeAnomalyClimatology(idata, DateArr)
        idata_anom = anomalies.reshape([shape[0]*shape[1],shape[2]]).T
    else:
        anomalies = idata[:,:,:]
        idata_anom = idata.reshape([shape[0]*shape[1],shape[2]]).T
         
    xdata = np.arange(len(DateArr))  
    
    Ntrue = len(DateArr)
    alpha = 1 - CL
    
    model_ols =  linear_model.LinearRegression()
    model_ols.fit(xdata.reshape(-1, 1),idata_anom) 
    idata_anom_trend = model_ols.coef_[:,0] 

    SignificanceArray = np.zeros([shape[0]*shape[1]])
    for i in range(len(idata_anom_trend)):
        #if np.ma.is_masked(idata_anom[0,i]): continue
        y_fit = model_ols.intercept_[i] + xdata[:]*idata_anom_trend[i]
        regression_residual = idata_anom[:,i] - y_fit[:]
        r = np.corrcoef(regression_residual[1:],regression_residual[:-1])[0,1]
        #Neff = Ntrue*(1-r)/(1+r)
        Neff = EffectiveSampleSize(regression_residual,ac=ac)
        residual_variance = 1/(Neff)*np.sum(regression_residual[:]**2.)
        StandardError = np.sqrt(residual_variance/(np.sum((xdata[:] - np.mean(xdata[:]))**2.))) 
        
        df = Neff - 2
        tvalue = stats.t.ppf(1. - alpha/2.,df)
        beta_0 = 0. # i.e. the null hypothesis is that x and y are uncorrelated -> no trend
        tstatistic = np.abs((idata_anom_trend[i] - beta_0)/StandardError)
        SignificanceArray[i] = (0 if tstatistic < tvalue else 1)
        
    trend = idata_anom_trend.reshape([shape[0],shape[1]])*factor
    significance = SignificanceArray.reshape([shape[0],shape[1]])
       
    anomalies_out = np.ma.zeros(full_shape)
    anomalies_out.mask = [True]
    
    trend_out = np.ma.zeros([full_shape[0],full_shape[1]])
    trend_out.mask = [True]
    
    significance_out = np.ma.zeros([full_shape[0],full_shape[1]])
    significance_out.mask = [True]   
    
    anomalies_out[index] = anomalies[:,0,:]
    anomalies_out.mask[index] = False
    
    trend_out[index] = trend[:,0]
    trend_out.mask[index] = False
    
    significance_out[index] = significance[:,0]
    significance_out.mask[index] = False 
       
    if ano == False:
        return anomalies_out, trend_out, significance_out
    else:
        return trend_out, significance_out
    
    
def EffectiveSampleSize(sample,ac='ar1',alpha_th=0.1,CL=0.95):
	import numpy as np
	import scipy.stats as stats

	Ntrue = len(sample)
	if ac == 'ar1':
		alpha = np.corrcoef(sample[1:],sample[:-1])[0,1]
		Neff = Ntrue*(1-alpha)/(1+alpha)
		return Neff
		
	elif ac == 'full':
		alpha = []
		for lag in range(1,Ntrue-1):
			alpha += [np.corrcoef(sample[lag:],sample[:-lag])[0,1]]
		
		alpha = np.array(alpha)
		
		sigma_k_squared = [1/Ntrue]
		for i in range(1,len(alpha)):
			sigma_k_squared += [1/Ntrue*(1+2*np.sum(alpha[:i]**2.))]
			
		sigma_k_squared = np.array(sigma_k_squared)
		
		alpha_cl = 1 - CL
		tval = stats.t.ppf(1. - alpha_cl/2.,Ntrue-1)
		
		alpha_signif = np.copy(alpha)
		sel_m = 0
		for m in range(len(alpha)-2):
			if alpha[m+1] < 0 and alpha[m+1] + alpha[m+2] < 0:
				sel_m = m + 1
				break
		
		for m in range(sel_m):
			if tval*np.sqrt(sigma_k_squared[m]) > alpha[m] > -tval*np.sqrt(sigma_k_squared[m]):
				alpha[m] = 0.
		
		Neff = Ntrue/(1+2*np.sum(alpha[:sel_m]))

		return Neff
	else:
		print('ERROR! Wrong type given!')
		return Ntrue
    
def ComputeAnomalyClimatology(idata,DateArr,refdate = [None], dfmt='%Y-%m-%d',print_info=False):
	# refdate as vector including start and enddate as str, e.g.: ['2000-01-15','2009-12-15']
	"""
	Input: idata,DateArr,refdate = [None], dfmt='%Y-%m-%d'
	Output: anomaly, climatology
	"""

	import numpy as np
	import datetime
	nlat = idata.shape[0]
	nlon = idata.shape[1]
	ntime = idata.shape[2]
	anomaly = np.ma.zeros(idata.shape)
	climatology = np.ma.zeros([nlat,nlon,12])
	iorder = np.arange(12)

	
	if refdate[0] == None:
		# if no reference period is given

		if DateArr[0].month != 1:
			iorder = np.roll(iorder,-(12-DateArr[0].month+1))

		for i in range(12):
			if print_info: print(f'Compute climatology {i}/12.')
			climatology[:,:,i] = np.nanmean(idata[:,:,iorder[i]::12],axis=2)

		if print_info: print('')
		for i in range(ntime):
			if print_info: print(f'Compute anomaly {i}/{ntime}.')
			anomaly[:,:,i] = idata[:,:,i] - climatology[:,:,DateArr[i].month-1]
		
	else:
		# computes climatology with respect to a reference period
		# anomalies relative to reference climatology, but for the whole period given by 'DateArr'

		if all([type(i) == str for i in refdate]):
			ref_sdate = datetime.datetime.strptime(refdate[0],dfmt)
			ref_edate = datetime.datetime.strptime(refdate[1],dfmt)
		else:
			ref_sdate = refdata[0]
			ref_edate = refdata[1]

		si = 0
		ei = -1
		for i in range(ntime):
			if DateArr[i] == ref_sdate:
				si = i
			if DateArr[i] == ref_edate:
				ei = i+1

		ref_data = idata[:,:,si:ei]
		ref_date = DateArr[si:ei]
		print(f' -- reference date: {ref_date[0]} to {ref_date[-1]}')
				
		if ref_date[0].month != 1:
			iorder = np.roll(iorder,-(12-ref_date[0].month+1))	
		for i in range(12):
			climatology[:,:,i] = np.nanmean(ref_data[:,:,iorder[i]::12],axis=2)	
		
		for i in range(ntime):
			anomaly[:,:,i] = idata[:,:,i] - climatology[:,:,DateArr[i].month-1]

	return anomaly, climatology

# Linearized Model

In [3]:
def Linearized_Model(rhoao, ws10m, deltaQ, deltaT, no_coeff = True, Cx_SLHF=0.001, Cx_SSHF=0.001, print_vals=False):
	"""
	input: rhoao, ws10m, deltaQ, deltaT, no_coeff = True, Cx_SLHF=0.001, Cx_SSHF=0.001, print_vals=True
	output: lin_slhf, lin_sshf, simple_slhf, simple_sshf, Cx_SLHF, Cx_SSHF
	""" 

	import numpy as np
    
	g = 9.80665
	cp = 1004.7090 
	Lv = 2.5008e6
	zn = 10.

	ws_mean = np.mean(ws10m)
	ws_prim = ws10m - ws_mean

	rho_mean = np.mean(rhoao)
	rho_prim = rhoao - rho_mean

	dq_mean = np.mean(deltaQ)
	dq_prim = deltaQ - dq_mean

	t_mean = np.mean(deltaT)
	t_prim = deltaT - t_mean

	if print_vals:
		print(f'Wind mean, std: ({ws_mean:.2e}, {np.std(ws_prim):.2e})')  
		print(f'   Q mean, std: ({dq_mean:.2e}, {np.std(dq_prim):.2e})') 
		print(f'   T mean, std: ({t_mean:.2e}, {np.std(t_prim):.2e})') 
    
    # SLHF
	linear_part =    ws_mean*rho_mean*dq_mean + ws_mean*rho_mean*dq_prim + ws_mean*rho_prim*dq_mean + ws_prim*rho_mean*dq_mean
	nonlinear_part = ws_mean*rho_prim*dq_prim + ws_prim*rho_mean*dq_prim + ws_prim*rho_prim*dq_mean + ws_prim*rho_prim*dq_prim

	if no_coeff:
		lin_slhf =    Lv*(linear_part)
		nonlin_slhf = Lv*(linear_part + nonlinear_part)	
	else:
		lin_slhf =    Lv*Cx_SLHF*(linear_part)
		nonlin_slhf = Lv*Cx_SLHF*(linear_part + nonlinear_part)

    # SSHF
	linear_part_sh =    ws_mean*rho_mean*t_mean + ws_mean*rho_mean*t_prim + ws_mean*rho_prim*t_mean + ws_prim*rho_mean*t_mean
	nonlinear_part_sh = ws_mean*rho_prim*t_prim + ws_prim*rho_mean*t_prim + ws_prim*rho_prim*t_mean + ws_prim*rho_prim*t_prim

	lin_gz_part = g*zn*(ws_mean*rho_mean + ws_mean*rho_prim  + ws_prim*rho_mean)
	nonlin_gz_part = g*zn*(ws_prim*rho_prim)

	if no_coeff:
		lin_sshf =    cp*(linear_part_sh) + lin_gz_part
		nonlin_sshf = cp*(linear_part_sh + nonlinear_part_sh) + nonlin_gz_part
	else:
		lin_sshf =    cp*Cx_SSHF*(linear_part_sh) +  Cx_SSHF*lin_gz_part
		nonlin_sshf = cp*Cx_SSHF*(linear_part_sh + nonlinear_part_sh) + Cx_SSHF*nonlin_gz_part
    
	return lin_slhf, lin_sshf, nonlin_slhf, nonlin_sshf, Cx_SLHF, Cx_SSHF



def Compute_TransferCoefficient(RefFlux,NonLinModel_woCx,LinModel_woCx,multiplier=1,print_info=True):
	"""
	input: RefFlux,NonLinModel_woCx,LinModel_woCx
	output: NonLinModel_adjusted, LinModel_adjusted
	"""
	import numpy as np
	from sklearn import linear_model

	# Compute reference trend (i.e. the target trend)
	model_ols =  linear_model.LinearRegression() 
	time = np.arange(len(RefFlux)).reshape(-1,1)
	model_ols.fit(time,RefFlux)
	reference_trend = model_ols.coef_[0]

	# Compute trend of non-linear model
	time = np.arange(len(NonLinModel_woCx)).reshape(-1,1)
	model_ols.fit(time,NonLinModel_woCx)
	model_trend = model_ols.coef_[0]	

	# Compute trend of linear model
	time = np.arange(len(LinModel_woCx)).reshape(-1,1)
	model_ols.fit(time,LinModel_woCx)
	lin_model_trend = model_ols.coef_[0]	

	Cx = reference_trend/model_trend
	NonLinModel_adjusted = Cx * NonLinModel_woCx
	LinModel_adjusted = Cx * LinModel_woCx

	if print_info:
		print(f' Computed transfer coefficient: {Cx:.8f}')
		print(f'  Reference trend: {reference_trend*multiplier:.5f}')
		print(f'lin+non-lin model: {model_trend*Cx*multiplier:.5f}')
		print(f'     linear model: {lin_model_trend*Cx*multiplier:.5f}')

	return NonLinModel_adjusted, LinModel_adjusted, Cx

# Compute trend

In [4]:
def compute_trend(DateArr,idata,factor=120,print_trend=True):
	"""
	Input: DateArr,idata,factor=120,print_trend=True
	Output: model_ols.coef_[0]*factor, model_ols.intercept_
	"""
    
	from sklearn import linear_model
	import numpy as np
	
	ndims = idata.ndim
	shape = idata.shape
	model_ols =  linear_model.LinearRegression() 
	time = np.arange(len(DateArr)).reshape(-1,1)

	if ndims == 3 and shape[2] == len(DateArr):
		#idata = idata.reshape([shape[0]*shape[1],len(DateArr)]).T
		
		mask = idata.mask[:,:,0]		# wähle maske basierend auf 1. zeitschritt aus
		index = np.where(mask == False)	# extrahiere alle indizes, welche nicht maskiert sind,
		idata = idata[index].T			# reduziere daten auf ausgewählte indizes. letzte dimension (zeit) bleibt erhalten
		
	elif ndims == 2 and shape[1] == len(DateArr):
		idata = idata.T

	model_ols.fit(time, idata) 

	if ndims == 3:
		trend_out = np.ma.zeros([shape[0],shape[1]])	# erstelle globales gitter, aber mit maske
		trend_out.mask = [True]							# notwendig weil np.ma.zeros kein 'mask=' argument hat
		trend_out.mask[index] = False					# entferne maske von allen gitterpunkten, die verwendet werden
		trend_out[index] = model_ols.coef_[:,0]*factor	# füge daten in output array ein
    	
		interc_out = np.ma.zeros([shape[0],shape[1]])
		interc_out.mask = [True]
		interc_out.mask[index] = False
		interc_out[index] = model_ols.intercept_*factor    	
	
		return trend_out, interc_out
		#return model_ols.coef_.reshape([shape[0],shape[1]])*factor, model_ols.intercept_.reshape([shape[0],shape[1]])
	elif ndims == 2: 
		return model_ols.coef_.reshape([shape[0]])*factor, model_ols.intercept_.reshape([shape[0]])
	elif ndims == 1: 
		if print_trend: print(f'{model_ols.coef_[0]*factor:.4f}')
		return model_ols.coef_[0]*factor, model_ols.intercept_

In [7]:
def TS_ApplyMovingAverage(idata=[],window=12):
	"""
	input:  idata,window=12 (i.e., total window size -> +-window/2)
	output: odata (smoothed timeseries)
	"""

	from sys import path
	import numpy as np

	if len(idata) == 0:
		print(' Input: idata')
		print('Output: smoothed idata')
		return

	rmw = np.ones(window)
	ndata = len(idata)
	odata = np.ma.zeros(ndata)

	for i in range(ndata):					# loop over all DATA 
		switch0 = 1					# switch: 
		for j in range(len(rmw)):			# loop over all WEIGHTS
			rmi = i - (len(rmw)-1)//2 + j		# running mean index: data index (i) minus half number of weights plus weight index (j)
											# asymmetrisch: dh bei len(rmw)==12 werden die 5 Monate vor und 6 Monate nach Zielmonate verw.
			if rmi < 0: continue			# if index is smaller than 0, continue to next weight. important for left boundary
			if rmi > ndata-1: continue		# important for right boundary
			odata[i] = odata[i] + idata[rmi]*rmw[j]
			index1 = j							# index of the last weight that is used
			if switch0 == 1:					
				index0 = j						# index of the first weight
				switch0 = 0						# turn off switch
		odata[i] = odata[i]/np.sum(rmw[index0:index1+1])	# divide by sum of the weights

	MC = np.ma.array([0],mask=[True])[0]
	odata[:window//2] = MC #np.NaN
	odata[-window//2:] = MC #np.NaN
	#print(odata)
	return odata

# Compute OHT trend

In [ ]:
def CreateDateArray(sdate,edate,resol='monthly', dfmt='%Y-%m-%d'):
	# 
	"""
	 INPUT: sdate,edate,resol='monthly', dfmt='%Y-%m-%d'
	 		(possible resolutions: 'hourly', 'daily', 'monthly', 'yearly')
	OUTPUT: dateArr
	"""
	import datetime
	from inspect import currentframe, getframeinfo

	if type(sdate) == str:
		sdate = datetime.datetime.strptime(sdate,dfmt)
	if type(edate) == str:
		edate = datetime.datetime.strptime(edate,dfmt)	

	syear = sdate.year
	smonth = sdate.month
	sday = sdate.day
	eyear = edate.year
	emonth = edate.month
	eday = edate.day

	dateArr = []
	if resol == 'hourly':
		dt = datetime.timedelta(hours=1)
		date = sdate
		while date <= edate:
			dateArr += [date]
			date = date + dt

	elif resol == 'daily':
		for year in range(syear,eyear+1):
			DaysPerMonths = [31,28,31,30,31,30,31,31,30,31,30,31]
			if year%4 == 0: DaysPerMonths[1] = 29

			smonth_iter = 1
			emonth_iter = 13
			if year == syear:
				smonth_iter = smonth
			if year == eyear:
				emonth_iter = emonth + 1

			for month in range(smonth_iter,emonth_iter):
				sday_iter = 1
				eday_iter = DaysPerMonths[month-1] + 1
				if year == syear and month == smonth:
					sday_iter = sday
				if year == eyear and month == emonth:
					eday_iter = eday + 1

				for day in range(sday_iter,eday_iter):
					dateArr += [datetime.datetime(year,month,day)]

	elif resol == 'monthly':
		for year in range(syear,eyear+1):
			smonth_iter = 1
			emonth_iter = 13
			if year == syear:
				smonth_iter = smonth
			if year == eyear: 
				emonth_iter = emonth + 1
			for month in range(smonth_iter,emonth_iter):
				dateArr += [datetime.datetime(year,month,sday)]	
	elif resol == 'yearly':
		for year in range(syear,eyear+1):
			dateArr += [datetime.datetime(year,smonth,sday)]
	else:
		frameinfo = getframeinfo(currentframe())
		print(f'ERROR! Wrong resolution selected. Use one of the following: hourly/daily/monthly/yearly ({_currentroutine}:line {frameinfo.lineno+2}).')
		return

	return dateArr


def TS_Compute_Anomaly(idata,date,x_reftime = [None]):
	import numpy as np
	import datetime
	ndata = len(idata)
	odata_anom = np.ma.zeros(ndata)
	odata_clim = np.ma.zeros(12)
	iorder = np.arange(12)

	if x_reftime[0] == None:
		if date[0].month != 1:
			iorder = np.roll(iorder,-(12-date[0].month+1))

		for i in range(12):
			odata_anom[i::12] = idata[i::12] -  np.nanmean(idata[i::12])
			odata_clim[i] = np.nanmean(idata[iorder[i]::12])
	else:

		ref_clim = np.ma.zeros(12)
		ref_sdate = datetime.datetime(x_reftime[0],x_reftime[1],date[0].day)
		ref_edate = datetime.datetime(x_reftime[2],x_reftime[3],date[0].day)

		si = 0
		ei = -1
		for i in range(len(idata)):
			if date[i] == ref_sdate:
				si = i
			if date[i] == ref_edate:
				ei = i+1

		ref_data = idata[si:ei]
		ref_date = date[si:ei]


		if ref_date[0].month != 1:
			iorder = np.roll(iorder,-(12-ref_date[0].month+1))	
		for i in range(12):
			ref_clim[i] = np.nanmean(ref_data[iorder[i]::12])	
		
		for i in range(len(idata)):
			odata_anom[i] = idata[i] - ref_clim[date[i].month-1]

		odata_clim = np.copy(ref_clim)
	
	return odata_anom, odata_clim


def TS_Compute_CI(idata, DateArr, CL = 0.95, ac='ar1', return_vals = False, print_vals = True, return_01 = False):
    """
    - idata as original time series
    - computes confidence intervals for the trend based on anomalies using a two-tailed t-test
    - effective sample size (auto-correlation) is considered for computing degrees of freedom and residual variance
    - t-test checks whether the trend deviates from 0 (no trend) or not
    - default confidence level (CL) is 95 %
    """
    import numpy as np
    import scipy.stats as stats
    from sklearn import linear_model
    
    data_anom = np.copy(idata)

    xdata = np.arange(len(DateArr))  
    
    model_ols =  linear_model.LinearRegression()
    model_ols.fit(xdata.reshape(-1, 1),idata_anom) 
    idata_anom_trend = model_ols.coef_[0]   

    y_fit = model_ols.intercept_ + xdata*idata_anom_trend
    
    ### regression residual
    regression_residual = idata_anom - y_fit
    
    ### autocorrelation of regr.res.
    r = np.corrcoef(regression_residual[1:],regression_residual[:-1])[0,1]
    Ntrue = len(idata)
    Neff = EffectiveSampleSize(regression_residual,ac=ac)

    ### Standard error
    residual_variance = 1/(Neff)*np.sum(regression_residual**2.)
    StandardError = np.sqrt(residual_variance/(np.sum((xdata - np.mean(xdata))**2.)))
#    if print_vals: print(f' -- Standard error = {StandardError:.2f}')
    
    alpha = 1 - CL
    df = Neff - 2
    tvalue = stats.t.ppf(1. - alpha/2.,df)
    
    beta_0 = 0. # i.e. the null hypothesis is that x and y are uncorrelated -> no trend
    tstatistic = np.abs((idata_anom_trend - beta_0)/StandardError)
    trend_signif = ('NOT ' if tstatistic < tvalue else '')
    
    ### ALTERNATIVE COMPUTATION
    r_time = np.corrcoef(xdata,idata_anom)[0,1]
    tstatistic_estimate = np.abs(r_time*np.sqrt(Neff)/np.sqrt(1. - r_time**2))
    CI_lower_estimate = stats.norm.ppf(alpha/2., loc=idata_anom_trend, scale=StandardError)
    CI_upper_estimate = stats.norm.ppf(1. - alpha/2., loc=idata_anom_trend, scale=StandardError)

    CI_upper = idata_anom_trend + tvalue*StandardError
    CI_lower = idata_anom_trend - tvalue*StandardError
    
    if print_vals: print(f' -- (t value,t stat., t stat. est.) = ({tvalue:.1f},{tstatistic:.1f},{tstatistic_estimate:.1f})')
    if print_vals: print(f' --    Confidence intervals = ({CI_upper:.3f},{CI_lower:.3f})')
    if print_vals: print(f' -- Alternative estimate CI = ({CI_upper_estimate:.3f},{CI_lower_estimate:.3f})\n\n')        
     
    if print_vals: print(f'# Trend is {trend_signif}significant at CL = {CL*100:.1f} %.')        
    
    if return_vals and return_01:
    	return np.array([CI_upper,idata_anom_trend,CI_lower]), (0 if tstatistic < tvalue else 1)
    elif return_vals:
    	return np.array([CI_upper,idata_anom_trend,CI_lower])
    elif return_01: 
    	return (0 if tstatistic < tvalue else 1)
    else:
    	return



def ComputeOHTTrend(mbFS_domain,ohct_domain,DFBSO_pw_extended,corr_global_mean,\
                           sel_anomalies = True, nyrs = 5, csdate = '1950-01-15', cedate = '2019-12-15', no_ohct=False):
    # nyrs : number of avering years, for 5-year mean
    
    DateArr_cut = CreateDateArray('1950-01-15','2019-12-15')   
    breite_trend_oht = []
    breite_mean_absOHT = []
    breite_trend_mbfs = []
    breite_trend_ohct = []
    breite_area = []
    breite_diff_2decades = []
    breite_trend_5yr_mean = []

    for i in range(90-61,95,5):
        print(f'Latitude: {i}')
        # MASKING, MOVING AVERAGE, AREA MEAN 
        
        if no_ohct:
            diff_domain = mr.ImprovedMasking(mbFS_domain[...],lats,lons,amask=f'{90-i}N 90N 90W 30E')  
        else:
            diff_domain = mr.ImprovedMasking(mbFS_domain[...] - ohct_domain[...],lats,lons,amask=f'{90-i}N 90N 90W 30E')
        
        diff_domain_mean = mr.ComputeStats3D(diff_domain,lats,lons)[0]

        area = mr.ComputeStats(diff_domain[...,0],lats,lons)[4]
        breite_area += [[90-i,area]]

        indirect_transport = DFBSO_pw_extended - (diff_domain_mean - corr_global_mean)*area/1e15

        if sel_anomalies:
            indirect_transport, _ = TS_Compute_Anomaly(indirect_transport,DateArr_cut)

                
        # MEAN ABSOLUTE TRANSPORT
        mean_absOHT_upper, mean_absOHT_lower = CI_mean(indirect_transport,ac='full') #CI_diff_means(indirect_transport[-20:],indirect_transport[:20],ac='full')
        breite_mean_absOHT += [[90-i,mean_absOHT_upper,np.ma.mean(indirect_transport),mean_absOHT_lower]]   
        
        last_decade = np.ma.mean(indirect_transport[-240:])
        first_decade = np.ma.mean(indirect_transport[:240])
        dec_upper, dec_lower = CI_diff_means(indirect_transport[-240:],indirect_transport[:240],ac='full')

        breite_diff_2decades += [[90-i, dec_upper, last_decade-first_decade, dec_lower]]

        # TREND OVER 14 5-YEARS MEAN 
        tyrs = int(cedate[0:4]) - int(csdate[0:4]) + 1 # total years
        oht_5yr_mean = np.array([np.ma.mean(indirect_transport[i*nyrs*12:(i+1)*nyrs*12]) for i in range(int(tyrs/nyrs))])
        DateArr_5yr = CreateDateArray(csdate,cedate,resol='yearly')[::5]
        upper_5yr, trend_5yr, lower_5yr = TS_Compute_CI(oht_5yr_mean,DateArr_5yr,return_vals=True,ac='full',print_vals=False)*2
        breite_trend_5yr_mean += [[90-i,upper_5yr, trend_5yr, lower_5yr]]

        # MONTHLY TREND 
        upper, trend, lower = TS_Compute_CI(indirect_transport,DateArr_cut,return_vals=True,ac='full',print_vals=False)*120

        breite_trend_oht += [[90-i,upper, trend, lower]]

    breite_area = np.array(breite_area)
    breite_trend_oht = np.array(breite_trend_oht)
    breite_mean_absOHT = np.array(breite_mean_absOHT)
    breite_diff_2decades = np.array(breite_diff_2decades)
    breite_trend_5yr_mean = np.array(breite_trend_5yr_mean)
    
    return breite_area, breite_trend_oht, breite_mean_absOHT, breite_diff_2decades, breite_trend_5yr_mean


In [ ]:
area, oht_trend, _, _, oht_trend_5yr = ComputeMeridionalTrend(FS,OHCT,OHT_ArcticGateways,R_global)

# Compute mass stream function

In [9]:
def FindDateIndex(DateArr,date,dformat='%Y-%m-%d'):
    import numpy as np
    import datetime
    
    if isinstance(date,str):
        date = datetime.datetime.strptime(date,dformat)
    
    idx = np.where(np.array(DateArr) == date)[0][0]
    
    return idx

def ComputeStreamFunction(wind_component,levels,spatial_coords,target_axis=0,level_axis=2,time_axis=3):#,order=('lats','lons','levs','time')):
    import numpy as np
    
    REARTH = 6371e3
    GCONST = 9.80665
    
    resol = np.abs(spatial_coords[0]-spatial_coords[1])*(np.pi/180.)

    axis_no = np.arange(4)
    remainder = axis_no[np.isin(axis_no,[target_axis,level_axis,time_axis],invert=True)][0]
    permutation = (target_axis,remainder,level_axis,time_axis)
    
    #print(permutation)
            
    wind_component = np.transpose(wind_component,axes=permutation)
    
    shape = wind_component.shape
    #print(shape) #(lats,lons,level,time)
    
    if shape[2] != len(levels): 
        print(f'Invalid number of levels.')
        return False
    
    # temporal mean
    wind_tmn = np.ma.mean(wind_component,axis=3)
    
    # zonal/meridional mean
    wind_smn = np.ma.mean(wind_component,axis=(3,0))
    
    wind_ano = np.zeros_like(wind_tmn)
    wind_cui = np.zeros_like(wind_tmn)
    stream_f = np.ma.zeros([shape[0],shape[2]])
      
    for i in range(shape[0]):
        wind_ano[i,...] = wind_tmn[i,...] - wind_smn[...]
        
    wind_cui[...,0] = wind_ano[...,0]*levels[0]*100.
    
    for i in range(1,shape[2]):
        wind_cui[...,i] = wind_cui[...,i-1] + wind_ano[...,i]*(levels[i]-levels[i-1])*100.
    
    for j in range(shape[2]): # über levels (25)
        for i in range(shape[0]): # über lons (1440)
            stream_f[i,j] = np.ma.sum(wind_cui[i,:,j]*resol)
            
    return stream_f[...]*(REARTH/GCONST), wind_tmn, wind_smn, wind_ano, wind_cui


def PlotStreamFunction(sf,coords,level,nlines=10,figsize=(12,6),fnts = 18,limits=None,savefig=None,fmt='%.1f',plttitle=''):
    import numpy as np
    import matplotlib.pylab as plt
    from matplotlib import ticker
    
    if limits == None:
        lvl = np.arange(sf.min(),sf.max(),(sf.max()-sf.min())/nlines)
    else:
        lvl = np.arange(limits[0],limits[1],(limits[1]-limits[0])/nlines)

    plt.figure(figsize=figsize)
    ax = plt.subplot(1,1,1)
    CSp = plt.contour(coords,level,sf,levels=lvl[lvl>0],linewidths=1.0,linestyles='solid',colors='k')
    CS0 = plt.contour(coords,level,sf,levels=0,linewidths=3.0,linestyles='solid',colors='k')
    CSn = plt.contour(coords,level,sf,levels=lvl[lvl<0],linewidths=1.0,linestyles='dotted',colors='k')
    
    plt.clabel(CSp,lvl[lvl>0][1::2],inline=1,fmt=fmt,fontsize=10, use_clabeltext=True)
    plt.clabel(CS0,[0.],inline=1,fmt=fmt,fontsize=10, use_clabeltext=True)
    plt.clabel(CSn,lvl[lvl<0][1::2],inline=1,fmt=fmt,fontsize=10, use_clabeltext=True)
    

    plt.title(plttitle,loc='right',pad=10)

    plt.tick_params(axis='both',which='both',direction='out')
                   
    plt.xlim(np.min(coords),np.max(coords))
    #plt.ylim(np.max(level),np.min(level))
    plt.ylim(1000,0)
    print(np.min(coords),np.max(coords))
    plt.ylabel('hPa',fontsize=fnts)
    plt.xticks(fontsize=fnts)
    
    
    if np.array(coords).max() - np.array(coords).min() == 180:
        plt.xticks([90,60,30,0,-30,-60,-90],['90°N','60°N','30°N','0°','30°S','60°S','90°S'],fontsize=fnts)
        ax.xaxis.set_minor_locator(ticker.FixedLocator([80,70,50,40,20,10,-10,-20,-40,-50,-70,-80]))
    else:
        plt.xticks([0,90,180,270,360],['0°','90°E','180°','90°W','0°'],fontsize=fnts)
        
        
    plt.yticks(fontsize=fnts)
    
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    if savefig != None:
        plt.savefig(savefig,dpi=300,bbox_inches='tight',facecolor='white')
    
    plt.show()

In [ ]:
from netCDF4 import Dataset

ifile = '1950-2019-V3D-LL320.nc'
idata_v = Dataset(ifile, mode='r')

lats =  idata_v.variables['latitude'][:]
lons =  idata_v.variables['longitude'][:]
time =  idata_v.variables['time'][:]
level = idata_v.variables['level'][:]

time_obj = Convert_Hours_Since_Time(time,return_value=True)

sindex = FindDateIndex(time_obj,'1984-12-01')
eindex = FindDateIndex(time_obj,'2019-02-01')
nseason = 35


v_dask = da.from_array(idata_v.variables['v'], chunks=shape)

sf_seasonal = np.zeros([nseason,721,25])
for i in range(nseason):
    print(i)
    v_seasonal = np.mean(v_dask[sindex+i*3:sindex+(i+1)*3,...],axis=0)
    v_seasonal = v_seasonal.compute()
    sf, *_ = ComputeStreamFunction(v_seasonal[np.newaxis,...],level,lons,target_axis=2,level_axis=1,time_axis=0)
    sf_seasonal[i,...] = sf[...]